In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from config import g_key
from rossmo_et_ridgway import Rossmo

### Notes

* Ridgway notes need to be combined with csv sheets, methinks. 
    * https://docs.google.com/spreadsheets/d/1NHHZEL_3UaFRxgpGcsWyooXnLL-YStYilmb8TJ-JMC4/edit#gid=0
    
* max_distance() in rossmo_et_ridgway needs to exclude outliers, e.g. the max distance for ridgway is 160 miles, which gives a HUGE radius. 
* I also need to review how to scale the heatmap better -- perhaps normalize the output from 0 to 1. 

* Gmaps doc
    * https://jupyter-gmaps.readthedocs.io/en/latest/api.html

In [35]:
body_locations_path = '../resources/Ridgway/body_locations.csv'
disappearances_path = '../resources/Ridgway/disappearances.csv'
ridgway_locations_path = '../resources/Ridgway/ridgway_locations.csv'

df_victims = pd.read_csv(body_locations_path)
df_disappearances = pd.read_csv(disappearances_path)
df_ridgway = pd.read_csv(ridgway_locations_path)

df_disappearances['coordinates'] = list(zip(df_disappearances['Y'], df_disappearances['X']))

In [36]:
df_disappearances.sample(5)

,X,Y,Name,description,coordinates
20,-122.334371,47.605024,Kimi-Kai Pitsor,NaN,"(47.6050242, -122.3343709)"
33,-122.284051,47.478834,Patricia Michelle Barczak,NaN,"(47.4788341, -122.2840507)"
22,-122.277116,47.549709,Lisa Yates,NaN,"(47.5497095, -122.2771163)"
4,-122.284142,47.475779,Carrie Ann Rois,NaN,"(47.4757788, -122.2841416)"
41,-122.284259,47.473064,Tina Marie Thompson,NaN,"(47.4730643, -122.2842593)"


In [37]:
R = Rossmo(df_disappearances['coordinates'].to_list(), accuracy=100)

In [38]:
rossmo_results = R.rossmo_results

In [39]:
df_rossmo_results = pd.DataFrame({'coordinates': rossmo_results.keys(), 'score': rossmo_results.values()})

In [40]:
df_rossmo_results['score_normalized'] = (
    (df_rossmo_results['score'] - df_rossmo_results['score'].min()) / (df_rossmo_results['score'].max() - df_rossmo_results['score'].min()) 
) # * 100
df_rossmo_results.sample(5)

,coordinates,score,score_normalized
5880,"(47.605357968655355, -122.84830116793384)",55.764022,0.077297
5660,"(47.57701609367763, -122.53241312087444)",79.449376,0.170173
6413,"(47.69038359358855, -121.79007621028484)",54.022412,0.070468
539,"(46.854298281745486, -122.20073067146207)",52.191127,0.063287
2747,"(47.16605890650053, -122.32708589028583)",76.121243,0.157123


In [41]:
stats = df_rossmo_results['score'].describe()
stats

count    10000.000000
mean        57.238645
std         18.523103
min         36.051665
25%         45.808452
50%         52.087252
75%         62.081477
max        291.072206
Name: score, dtype: float64

In [42]:
# set normalized score for score values below q1 to 0.0
q1 = stats['25%']
df_rossmo_results.loc[df_rossmo_results['score'] <= q1, ['score_normalized']] = 0.0

In [43]:
df_rossmo_results.loc[df_rossmo_results['score_normalized'] > 0].sort_values(by='score_normalized', ascending=False).head(5)

,coordinates,score,score_normalized
4645,"(47.43530671878897, -122.29549708557988)",291.072206,1.000000
4944,"(47.47781953125557, -122.27970268322692)",218.596176,0.715803
4545,"(47.42113578130011, -122.29549708557988)",215.917752,0.705300
4445,"(47.40696484381124, -122.29549708557988)",196.452024,0.628970
4745,"(47.449477656277836, -122.29549708557988)",195.150791,0.623868


In [44]:
from mapping_helper_functions import convert_latitude_to_webmercator, convert_longitude_to_webmercator

df_rossmo_results[['latitude', 'longitude']] = pd.DataFrame(df_rossmo_results['coordinates'].tolist(), index=df_rossmo_results.index)

df_rossmo_results['latitude_webmercator'] = convert_latitude_to_webmercator(df_rossmo_results['latitude'])
df_rossmo_results['longitude_webmercator'] = convert_longitude_to_webmercator(df_rossmo_results['longitude'])

In [47]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON_RETINA, get_provider
from bokeh.models import HoverTool

# show map in notebook
output_notebook()

tile_provider = get_provider(CARTODBPOSITRON_RETINA)

x_range = df_rossmo_results['longitude_webmercator'].min(), df_rossmo_results['longitude_webmercator'].max()  # lon
y_range = df_rossmo_results['latitude_webmercator'].min(), df_rossmo_results['latitude_webmercator'].max() # lat

x_image_anchor = df_rossmo_results['longitude_webmercator'].min()
y_image_anchor = df_rossmo_results['latitude_webmercator'].min() 

# range bounds supplied in web mercator coordinates
p = figure(
    x_range=x_range, 
    y_range=y_range,
    x_axis_type='mercator', 
    y_axis_type='mercator',
    lod_threshold=None
)
p.add_tile(tile_provider)

d = df_rossmo_results['score_normalized'].to_numpy().reshape((100, 100))

p.image(
    image=[d], 
    x=x_range[0]+15700, # compensating by this amount centers the image as expected
    y=y_range[0]-800, # compensating by this amount centers the image as expected
    dw=abs(x_range[1] - x_range[0]), 
    dh=abs(y_range[1] - y_range[0]), 
    palette='Spectral11',  
    alpha=0.65
)

x_circle = df_rossmo_results.loc[df_rossmo_results['score_normalized'] == 1.0]['longitude_webmercator']
y_circle = df_rossmo_results.loc[df_rossmo_results['score_normalized'] == 1.0]['latitude_webmercator']

p.circle(
    x=x_circle,
    y=y_circle,
    size=7,
    color='black'
)

show(p)

Loading BokehJS ...

In [46]:
df_rossmo_results.sample(25)

,coordinates,score,score_normalized,latitude,longitude,latitude_webmercator,longitude_webmercator
2284,"(47.0952042190562, -122.91147877734572)",44.869580,0.000000,47.095204,-122.911479,5.957628e+06,-1.368244e+07
5738,"(47.59118703116649, -122.1849362691091)",98.097371,0.243297,47.591187,-122.184936,6.039110e+06,-1.360156e+07
2857,"(47.180229843989395, -122.48502991381552)",65.145695,0.114085,47.180230,-122.485030,5.971542e+06,-1.363497e+07
8221,"(47.94546046838813, -121.9164314291086)",49.655168,0.053343,47.945460,-121.916431,6.097786e+06,-1.357168e+07
6112,"(47.64787078112195, -121.77428180793187)",54.818340,0.073589,47.647871,-121.774282,6.048472e+06,-1.355585e+07
8276,"(47.94546046838813, -122.78512355852196)",47.140527,0.043482,47.945460,-122.785124,6.097786e+06,-1.366838e+07
3641,"(47.293597343900316, -122.232319476168)",88.778884,0.206757,47.293597,-122.232319,5.990129e+06,-1.360684e+07
1080,"(46.92515296918982, -122.84830116793384)",42.640997,0.000000,46.925153,-122.848301,5.929866e+06,-1.367541e+07
3087,"(47.20857171896712, -122.95886198440463)",46.436939,0.040723,47.208572,-122.958862,5.976185e+06,-1.368772e+07
8635,"(48.00214421834359, -122.13755306205019)",55.615572,0.076715,48.002144,-122.137553,6.107212e+06,-1.359629e+07
